In [5]:
import os 
import sys
sys.path.append('D:\Project\optical_film_toolbox_master\optical_film_toolbox')

import time
import pandas
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt 
import plotly

from models.AE import AE
from models.CVAE import CVAE

from common.DataLoader import MaterialLoader

In [6]:
WAVELENGTH_RANGE = (200, 1200)
INPUT_SHAPE = (WAVELENGTH_RANGE[1] - WAVELENGTH_RANGE[0], 3)
INPUT_LENGTH = WAVELENGTH_RANGE[1] - WAVELENGTH_RANGE[0]
LATENT_DIM = 4

materials_li = []
materials = MaterialLoader(wl_s=WAVELENGTH_RANGE[0]/1000, wl_e=WAVELENGTH_RANGE[1]/1000)
for sample in materials.load_total_material_generator():
    materials_li.append(sample)

2%|▏         | 17/925 [00:00<00:11, 81.18it/s]Load Ag_Windt filled
Material wavelength bound is out of range!
Load Al_Larruquert filled
Material wavelength bound is out of range!
  7%|▋         | 66/925 [00:00<00:07, 109.09it/s]Load GaAs_Kachare filled
Material wavelength bound is out of range!
Load InAs_Lorimor filled
Material wavelength bound is out of range!
Load CdGeAs2_Boyd-o filled
Material wavelength bound is out of range!
Load CdGeAs2_Boyd-e filled
Material wavelength bound is out of range!
Load Au_Windt filled
Material wavelength bound is out of range!
 11%|█         | 103/925 [00:00<00:08, 92.77it/s]Load B_Fernandez-Perea filled
Material wavelength bound is out of range!
 13%|█▎        | 124/925 [00:01<00:09, 86.69it/s]Load C_Dore filled
Material wavelength bound is out of range!
 16%|█▌        | 149/925 [00:01<00:10, 70.80it/s]Load SiC_Fischer-o filled
Material wavelength bound is out of range!
Load SiC_Fischer-e filled
Material wavelength bound is out of range!
Load CaMg(CO

In [14]:
def material_data_generator():
    for material in materials_li:
        print(material["Ag_Johnson"][0])

        yield material[1][0]

material_data_generator()

<generator object material_data_generator at 0x000002D2106C4048>

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(TRAIN_BUF).batch(BATCH_SIZE)